<a href="https://colab.research.google.com/github/ayanban011/Getting-Started_with_Pytorch/blob/main/Bag_of_Words_and_Tf_IDF_representations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#loading the requirements
!pip install -r https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/requirements.txt
!wget -q https://raw.githubusercontent.com/MicrosoftDocs/pytorchfundamentals/main/nlp-pytorch/torchnlp.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch
import torchtext
import os
import collections
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


train.csv: 29.5MB [00:00, 108MB/s]
test.csv: 1.86MB [00:00, 81.2MB/s]                  


Building vocab...
Vocab size =  95812


## **Bag of Words text representation**
Because words represent meaning, sometimes we can figure out the meaning of a text by just looking at the individual words, regardless of their order in the sentence. For example, when classifying news, words like weather, snow are likely to indicate weather forecast, while words like stocks, dollar would count towards financial news.

**Bag of Words (BoW)** vector representation is the most commonly used traditional vector representation. Each word is linked to a vector index, vector element contains the number of occurrences of a word in a given document.

In [3]:
#generating bag of words
def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(f"sample text:\n{train_dataset[0][1]}")
print(f"\nBoW vector:\n{to_bow(train_dataset[0][1])}")

sample text:
Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.

BoW vector:
tensor([0., 0., 2.,  ..., 0., 0., 0.])


> **Note:** Here we are using global `vocab_size` variable to specify default size of the vocabulary. Since often vocabulary size are pretty big, we can limit the size of the vocabulary to most frequent words. Try lowering `vocab_size` value and running the code below, and see how it affects the accuracy. You should expect some accuracy drop, but not dramatic, in lieu of higher performance.

## **Training BoW classifier**

Now that we have learned how to build a Bag-of-Words representation of our text, let's train a classifier on top of it. First, we need to convert our dataset for training in such a way, that all positional vector representations are converted to bag-of-words representation. This can be achieved by passing `bowify` function as `collate_fn` parameter to standard torch `DataLoader`.  The `collate_fn` gives you the ability to apply your own function to the dataset as it's loaded by the Dataloader:

In [4]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Now let's define a simple classifier neural network that contains one linear layer. The size of the input vector equals to `vocab_size`, and the output size corresponds to the number of classes (4). Because we are solving a classification task, the final activation function is `LogSoftmax()`.

In [5]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Now we will define a standard PyTorch training loop. Because our dataset is quite large, for our teaching purpose we will train only for one epoch, and sometimes even for less than an epoch (specifying the `epoch_size` parameter allows us to limit training). We would also report accumulated training accuracy during training; the frequency of reporting is specified using `report_freq` parameter.

In [6]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

Let's see how the classifier performs on the training dataset.

In [7]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8046875
6400: acc=0.84328125
9600: acc=0.8558333333333333
12800: acc=0.86390625


(0.025767816409373333, 0.8685367803837953)

> The Bag-of-Words approach can be used in the same manner with n-gram tokenizer - only that the vocabulary size would be bigger, and thus the network would have too many parameters. In the next unit, we will see how bigram representation can be used together with embeddings.


## **Term Frequency / Inverse Document Frequency:  TF-IDF**

In BoW representation, word occurrences are evenly weighted, regardless of the word itself. However, it is clear that frequent words, such as *'a'*, *'in'*, *'the'* etc. are much less important for the classification, than specialized terms. In fact, in most NLP tasks some words are more relevant than others.

**TF-IDF** stands for **term frequency–inverse document frequency**. It is a variation of bag of words, where instead of a binary 0/1 value indicating the appearance of a word in a document, a floating-point value is used, which is related to the **_frequency of word occurrence_** in the corpus.

The formula to calculate TF-IDF is:  $w_{ij} = tf_{ij}\times\log({N\over df_i})$

Here's the meaning of each parameter in the formula:
* $i$ is the word 
* $j$ is the document
* $w_{ij}$ is the weight or the importance of the word in the document
* $tf_{ij}$ is the number of occurrences of the word $i$ in the document $j$, i.e. the BoW value we have seen before
* $N$ is the number of documents in the collection
* $df_i$ is the number of documents containing the word $i$ in the whole collection

TF-IDF value $w_{ij}$ increases proportionally to the number of times a word appears in a document and is offset by the number of documents in the corpus that contains the word, which helps to adjust for the fact that some words appear more frequently than others. For example, if the word appears in *every* document in the collection, $df_i=N$, and $w_{ij}=0$, and those terms would be completely disregarded.

First, let's compute document frequency $df_i$ for each word $i$. We can represent it as tensor of size `vocab_size`. We will limit the number of documents to $N=1000$ to speed up processing. For each input sentence, we compute the set of words (represented by their numbers), and increase the corresponding counter:

In [8]:
N = 1000
df = torch.zeros(vocab_size)
for _,line in train_dataset[:N]:
    for i in set(encode(line)):
        df[i] += 1
print(df)

tensor([  0.,   0., 982.,  ...,   0.,   0.,   0.])


Now that we have document frequencies for each word, we can define `tf_idf` function that will take a string, and produce TF-IDF vector. We will use `to_bow` defined above to calculate term frequency vector, and multiply it by inverse document frequency of the corresponding term. Remember that all tensor operations are element-wise, which allows us to implement the whole computation as a tensor formula:

In [9]:
def tf_idf(s):
    bow = to_bow(s)
    return bow*torch.log((N+1)/(df+1))

print(tf_idf(train_dataset[0][1]))

tensor([0.0000, 0.0000, 0.0363,  ..., 0.0000, 0.0000, 0.0000])


> You may have noticed that we used a slightly different formula for TF-IDF, namely $\log({N+1\over df_i+1})$ instead of $\log({N\over df_i})$. This yields similar results, but prevents division by 0 in those cases when $df_i=0$.

Even though TF-IDF representation calculates different weights to different words according to their importance, it is unable to correctly capture the meaning, largely because the order of words in the sentence is still not taken into account. As the famous linguist J. R. Firth said in 1935, “The complete meaning of a word is always contextual, and no study of meaning apart from context can be taken seriously”. We will learn in the later units how to capture contextual information from text using language modeling.
